# Project Overview 

Predicting depending on the car accident how severe will your injury be.

## The area's in which I look to address are:

How can this be used in the outside world?

What features did I use?

What did the visualizations tell about my data?

What does the statistical techniques conclude about my data?

What model was best fit?

## Gathered Data 
Gathered Traffic Crashes dataset from Chicago data portal


# Importing libraries

In [1]:
import pandas as pd
import sqlite3 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import make_scorer, accuracy_score 
from matplotlib import pyplot as plt
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
import seaborn as sns 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics 
from sklearn.utils import resample
from IPython.core.interactiveshell import InteractiveShell
from sklearn.model_selection import GridSearchCV
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 200)
from sklearn import metrics
%matplotlib inline

/Users/jdanielkim123/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [ ]:
#Loading and concating data
people = pd.read_csv('Traffic_Crashes_-_People.csv')
df = pd.read_csv('Traffic_Crashes_-_Crashes.csv')
result=pd.merge(people, df, on=['CRASH_RECORD_ID'], how = 'inner')

In [ ]:
#Initial cleaning
result.drop_duplicates(subset='CRASH_RECORD_ID', inplace=True)
A = ['REPORT_TYPE','INJURIES_TOTAL','RD_NO_y','CRASH_DATE_x','PERSON_ID','LOCATION','RD_NO_x','VEHICLE_ID','CITY','STATE','DRIVERS_LICENSE_STATE','SEAT_NO','DRIVERS_LICENSE_CLASS','HOSPITAL','EMS_AGENCY','EMS_RUN_NO','PEDPEDAL_ACTION','PEDPEDAL_VISIBILITY','PEDPEDAL_LOCATION','CELL_PHONE_USE','CRASH_DATE_EST_I','LANE_CNT','INTERSECTION_RELATED_I','NOT_RIGHT_OF_WAY_I','BAC_RESULT VALUE','HIT_AND_RUN_I','PHOTOS_TAKEN_I','STATEMENTS_TAKEN_I','DOORING_I','WORK_ZONE_I','WORK_ZONE_TYPE', 'WORKERS_PRESENT_I']     
result.drop(A,axis=1,inplace=True)
result.AGE.fillna(36,inplace=True)
result.SEX.fillna('U',inplace=True)
result['SEX'] =np.where(result['SEX']== 'U','M',result['SEX'])
result['SEX'] =np.where(result['SEX']== 'X','F',result['SEX'])
B = ['LATITUDE','LONGITUDE','MOST_SEVERE_INJURY',
 'INJURIES_FATAL',
 'INJURIES_INCAPACITATING',
 'INJURIES_NON_INCAPACITATING',
 'INJURIES_REPORTED_NOT_EVIDENT',
 'INJURIES_NO_INDICATION',
 'INJURIES_UNKNOWN', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'STREET_NO',
 'STREET_DIRECTION',
 'STREET_NAME', 'DATE_POLICE_NOTIFIED', 'CRASH_DATE_y']
result.drop(B,axis=1,inplace=True)
result.dropna(axis=0, inplace=True)
result['AGE']=result['AGE'].map(lambda x: int(x))
result['AGE']=result['AGE'].map(lambda x: str(x))
result['AGE']=result['AGE'].map(lambda x: x.replace('-49','49'))
result['AGE']=result['AGE'].map(lambda x: int(x))
result.drop(['CRASH_RECORD_ID'],axis=1,inplace=True)
result.drop(['ZIPCODE','ALIGNMENT'],axis=1,inplace=True)

# EDA

In [ ]:
#created variable name eda
eda =result

# Anova test
Test whether there is a statistically significant difference in the average age based on their injury classification
- H0: The average age between different injury classes are all the same
- HA: The average age between different injury classes are not the same

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
formula = 'AGE ~ C(INJURY_CLASSIFICATION)'
lm = ols(formula, eda).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

In [ ]:
#Plotting violin chart
violin, ax = plt.subplots(figsize=(10,8))
sns.violinplot(y=result['AGE'], x=result['INJURY_CLASSIFICATION'], data=result)
ax.set(title='Is there a significant difference in the average age based on injury classification?')
plt.xticks(rotation=45)
violin.savefig('violin.png', bbox_inches='tight');


In [ ]:
#creating a table to compare distribution of age and the injury class
no_indication = eda[eda['INJURY_CLASSIFICATION']=='NO INDICATION OF INJURY']
non_incapacitating = eda[eda['INJURY_CLASSIFICATION']=='NONINCAPACITATING INJURY']
reported = eda[eda['INJURY_CLASSIFICATION']=='REPORTED, NOT EVIDENT']
incapacitating = eda[eda['INJURY_CLASSIFICATION']=='INCAPACITATING INJURY']
fatal = eda[eda['INJURY_CLASSIFICATION']=='FATAL']
injury_bac = pd.DataFrame(columns=['NO INDICATION OF INJURY','NONINCAPACITATING INJURY','REPORTED, NOT EVIDENT','INCAPACITATING INJURY','FATAL'])
injury_bac['NO INDICATION OF INJURY']=no_indication.groupby('AGE')['AGE'].count()
injury_bac['NONINCAPACITATING INJURY']=non_incapacitating.groupby('AGE')['AGE'].count()
injury_bac['REPORTED, NOT EVIDENT']=reported.groupby('AGE')['AGE'].count()
injury_bac['INCAPACITATING INJURY']=incapacitating.groupby('AGE')['AGE'].count()
injury_bac['FATAL']=fatal.groupby('AGE')['AGE'].count()
injury_bac.fillna(0,inplace=True)
injury_bac

In [ ]:
injury_bac.reset_index(inplace=True)

In [ ]:
#distribution of age and injury that is incapacitating  
plt.scatter(injury_bac['AGE'], injury_bac['INCAPACITATING INJURY'])
plt.show();

In [ ]:
#distribution of age and injury that is fatal  
plt.scatter(injury_bac['AGE'], injury_bac['FATAL'])
plt.show();

People between their 20's to 40 are most likely to get an incapitating injury or fatal.

# Chi Squared

Is the type of injury independent of the crash type?
- H0: injury is independent 
- HA: injury is not independent

In [ ]:
#creating a table to compare distribution of crash type and the injury class
no_indication = eda[eda['INJURY_CLASSIFICATION']=='NO INDICATION OF INJURY']
non_incapacitating = eda[eda['INJURY_CLASSIFICATION']=='NONINCAPACITATING INJURY']
reported = eda[eda['INJURY_CLASSIFICATION']=='REPORTED, NOT EVIDENT']
incapacitating = eda[eda['INJURY_CLASSIFICATION']=='INCAPACITATING INJURY']
fatal = eda[eda['INJURY_CLASSIFICATION']=='FATAL']
injury_bac = pd.DataFrame(columns=['NO INDICATION OF INJURY','NONINCAPACITATING INJURY','REPORTED, NOT EVIDENT','INCAPACITATING INJURY','FATAL'])
injury_bac['NO INDICATION OF INJURY']=no_indication.groupby('FIRST_CRASH_TYPE')['FIRST_CRASH_TYPE'].count()
injury_bac['NONINCAPACITATING INJURY']=non_incapacitating.groupby('FIRST_CRASH_TYPE')['FIRST_CRASH_TYPE'].count()
injury_bac['REPORTED, NOT EVIDENT']=reported.groupby('FIRST_CRASH_TYPE')['FIRST_CRASH_TYPE'].count()
injury_bac['INCAPACITATING INJURY']=incapacitating.groupby('FIRST_CRASH_TYPE')['FIRST_CRASH_TYPE'].count()
injury_bac['FATAL']=fatal.groupby('FIRST_CRASH_TYPE')['FIRST_CRASH_TYPE'].count()
injury_bac.fillna(0,inplace=True)
injury_bac

In [ ]:
from scipy import stats
from scipy.stats import chi2_contingency
stat,p,dof,expected = stats.chi2_contingency(injury_bac)
prob=.95
critical = stats.chi2.ppf(prob,dof)
critical,stat

We reject the null. We further explored and the most leathel car accidents are ones that you run into a fixed object